In [40]:
%reload_ext autoreload
%autoreload 2

In [41]:
import requests
import pandas as pd
import numpy as np
import statcast
import os
import pathlib
import logging

# logging.basicConfig(level = logging.INFO)
pd.options.display.max_columns = 100

In [42]:
from injury.scrape.statsapi import scrape_il_data
from injury.scrape.prosports import scrape_dtd_data

### Query IL Data

In [128]:
start, end = 2012, 2022
status_changes, teams = scrape_il_data(start, end + 1)

INFO:injury.scrape.statsapi:Scraping IL data for 2012
INFO:injury.scrape.statsapi:Scraping IL data for 2013
INFO:injury.scrape.statsapi:Scraping IL data for 2014
INFO:injury.scrape.statsapi:Scraping IL data for 2015
INFO:injury.scrape.statsapi:Scraping IL data for 2016
INFO:injury.scrape.statsapi:Scraping IL data for 2017
INFO:injury.scrape.statsapi:Scraping IL data for 2018
INFO:injury.scrape.statsapi:Scraping IL data for 2019
INFO:injury.scrape.statsapi:Scraping IL data for 2020
INFO:injury.scrape.statsapi:Scraping IL data for 2021
INFO:injury.scrape.statsapi:Scraping IL data for 2022


In [130]:
if not os.path.exists("statsapi_data/"):
    os.mkdir("statsapi_data/")
status_changes.to_csv(f"statsapi_data/status_changes{start}-{end}.csv", index=False)
teams.to_csv(f"statsapi_data/teams{start}-{end}.csv", index=False)

### Query DTD Data

Takes a while so saves each year separately

In [131]:
scrape_dtd_data(2022, 2022 + 1, path="prosports_data/")

INFO:injury.scrape.prosports:Scraping DTD data for 2022


### Clean DTD Data

In [132]:
from injury.preprocess.prosports import ProsportsCleaner

In [133]:
# Read data
prosports = pd.concat(
    [pd.read_csv(f) for f in pathlib.Path("prosports_data").glob("prosports_*.csv")]
)
teams = pd.read_csv(f"statsapi_data/teams{start}-{end}.csv")

In [134]:
pc = ProsportsCleaner(prosports, teams)
dtd = pc.clean()

/Users/Sam/Desktop/Projects/injury/injury/preprocess/prosports.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_abbrev = prosports.name.str.contains("([A-Z]|r)\.")
/Users/Sam/Desktop/Projects/injury/injury/preprocess/prosports.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prosports["name"] = remove_accents(prosports["name"]).str.lower()
/Users/Sam/Desktop/Projects/injury/injury/preprocess/prosports.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

### Align DTD Data to MLB ids

In [135]:
from injury.preprocess.prosports import AlignProsportsMLB

In [136]:
db = statcast.db.Postgres()
players = db.query("player.sql", query_params={"min_year": 2012, "max_year": 2022})
players.to_parquet("../data/players.parquet")

/Users/Sam/opt/miniconda3/envs/injury/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [137]:
players = pd.read_parquet("../data/players.parquet")
apm = AlignProsportsMLB(dtd, players)
matched_dtd = apm.run()

In [140]:
# matched_dtd.shape

### Combine IL and DTD

In [144]:
status_changes = (
    pd.read_csv(f"statsapi_data/status_changes{start}-{end}.csv")
    .drop(columns=["resolutionDate", "id"])
    .rename(columns={"description": "notes"})
)

player_names = (
    players[["player_id", "full_name"]]
    .drop_duplicates("player_id")
    .rename(columns={"full_name": "name"})
)


il_df = status_changes[status_changes.notes.str.contains(r"the (\d+)(\s|-)day")]
il_df = il_df.merge(player_names, how="left")
il_df["dtd"] = False
matched_dtd["il_days"] = 0

/var/folders/6y/ttc306bd5rlc_xrt71tjyg7r0000gn/T/ipykernel_45553/2746471642.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  il_df = status_changes[status_changes.notes.str.contains(r'the (\d+)(\s|-)day')]


In [145]:
injuries = pd.concat([il_df, matched_dtd.drop(columns=["id"])])
injuries["date"] = pd.to_datetime(injuries["date"])
injuries["activated"] = injuries.notes.str.contains("activat")
injuries["transfer"] = injuries.notes.str.contains("transfer")

In [146]:
injuries.reset_index(drop=True).to_parquet(f"../data/injuries{start}-{end}.parquet")

### Scratch

In [89]:
import requests

start = "2015-01-01"
end = "2017-12-01"
url = "https://statsapi.mlb.com/api/v1/transactions?startDate={start}&endDate={end}"
results = requests.get(url.format(start=start, end=end))
results = results.json()["transactions"]

In [91]:
# results[0]

{'id': 213879,
 'person': {'id': 502462,
  'fullName': 'Ryan Kelly',
  'link': '/api/v1/people/502462'},
 'toTeam': {'id': 687,
  'name': 'Gigantes de Carolina',
  'link': '/api/v1/teams/687'},
 'date': '2015-01-01',
 'effectiveDate': '2015-01-01',
 'resolutionDate': '2015-01-01',
 'typeCode': 'ASG',
 'typeDesc': 'Assigned',
 'description': 'RHP Ryan Kelly assigned to Gigantes de Carolina.'}

In [92]:
[x for x in results if "person" in x and x["person"]["id"] == 502304]

[{'id': 213889,
  'person': {'id': 502304,
   'fullName': 'David Carpenter',
   'link': '/api/v1/people/502304'},
  'fromTeam': {'id': 144,
   'name': 'Atlanta Braves',
   'link': '/api/v1/teams/144'},
  'toTeam': {'id': 147,
   'name': 'New York Yankees',
   'link': '/api/v1/teams/147'},
  'date': '2015-01-01',
  'effectiveDate': '2015-01-01',
  'typeCode': 'TR',
  'typeDesc': 'Trade',
  'description': 'Atlanta Braves traded LHP Chasen Shreve and RHP David Carpenter to New York Yankees for LHP Manny Banuelos.'},
 {'id': 228619,
  'person': {'id': 502304,
   'fullName': 'David Carpenter',
   'link': '/api/v1/people/502304'},
  'toTeam': {'id': 147,
   'name': 'New York Yankees',
   'link': '/api/v1/teams/147'},
  'date': '2015-06-03',
  'effectiveDate': '2015-06-03',
  'resolutionDate': '2015-06-03',
  'typeCode': 'DES',
  'typeDesc': 'Designated for Assignment',
  'description': 'New York Yankees designated RHP David Carpenter for assignment.'},
 {'id': 229679,
  'person': {'id': 5023

In [12]:
# Align to prosports teams
teams.loc[teams.team == "ARI", "team_name"] = "Diamondbacks"

In [13]:
# Clean up prosports data
prosports.columns = prosports.columns.str.lower()
prosports.rename(columns={"team": "team_name"}, inplace=True)


# Merge
prosports["date"] = pd.to_datetime(prosports["date"])
prosports["year"] = prosports["date"].dt.year
prosports = prosports.merge(teams, on=["year", "team_name"], how="left")
prosports["activated"] = prosports["relinquished"].isnull()


prosports["name"] = np.where(
    prosports["acquired"].isnull(), prosports["relinquished"], prosports["acquired"]
)
prosports.drop(columns=["acquired", "relinquished"], inplace=True)
prosports["name"] = prosports["name"].str.replace(r"[\(\[].*?[\)\]]", "", regex=True)
prosports["name"] = prosports["name"].str.replace("•", "", regex=False)

# remove non-names & empty notes
prosports = prosports[
    prosports["name"].notnull() & prosports["name"].str.contains("[a-z]")
]
prosports = prosports[prosports["notes"].notnull()]

# replace abbrev
prosports.loc[prosports.name.str.contains("([A-Z]|r)\."), "name"] = prosports.loc[
    prosports.name.str.contains("([A-Z]|r)\."), "name"
].str.replace("\.", "", regex=True)
prosports["name"] = remove_accents(prosports["name"])

# lower
prosports["name"] = prosports["name"].str.lower()

# split names
prosports[["name", "name2"]] = prosports["name"].str.split("/", expand=True)
prosports["name"] = prosports["name"].str.strip()
prosports["name2"] = prosports["name2"].str.strip()

# dates
prosports["date"] = pd.to_datetime(prosports["date"])
prosports["dtd"] = prosports.notes.str.contains("DTD")

# id
prosports["id"] = list(prosports.index)

/var/folders/6y/ttc306bd5rlc_xrt71tjyg7r0000gn/T/ipykernel_13525/1141789034.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  prosports.loc[prosports.name.str.contains("([A-Z]|r)\."),"name"].str.replace("\.","", regex = True)
/var/folders/6y/ttc306bd5rlc_xrt71tjyg7r0000gn/T/ipykernel_13525/1141789034.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  prosports.loc[prosports.name.str.contains("([A-Z]|r)\."),"name"]= \


### Align MLB ids and names to DTD events

In [13]:
# dtd = prosports[prosports["dtd"]]

pc = ProsportsCleaner(prosports, teams)
dtd = pc.clean()

/Users/Sam/Desktop/Projects/injury/injury/preprocess/prosports.py:71: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_abbrev = prosports.name.str.contains("([A-Z]|r)\.")
/Users/Sam/Desktop/Projects/injury/injury/preprocess/prosports.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prosports["name"] = self._remove_accents(prosports["name"]).str.lower()
/Users/Sam/Desktop/Projects/injury/injury/preprocess/prosports.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [15]:
# db = statcast.db.Postgres()
# players = db.query("player.sql", query_params={"min_year":2012, "max_year":2021})
# players.to_parquet("players.parquet")

In [15]:
players = pd.read_parquet("../data/players.parquet")
# Normalize player names for matching
players["full_name"] = (
    players.name.apply(lambda x: x.split(", ")[1])
    + " "
    + players.name.apply(lambda x: x.split(", ")[0])
)
players["full_name"] = remove_accents(players["full_name"])

players.loc[players.full_name.str.contains("([A-Z]|r)\."), "full_name"] = players.loc[
    players.full_name.str.contains("([A-Z]|r)\."), "full_name"
].str.replace("\.", "", regex=True)

# remove middle initial
players.loc[
    players.full_name.str.contains("[A-Za-z]+ [A-Z] [A-Za-z]+"), "full_name"
] = players.full_name[
    players.full_name.str.contains("[A-Za-z]+ [A-Z] [A-Za-z]+")
].apply(
    lambda x: x.split(" ")[0] + " " + x.split(" ")[2]
)

players["lower_full_name"] = players["full_name"].str.lower()

In [19]:
from itertools import product

injured_players = players[
    players["lower_full_name"].isin(dtd["name"])
    | players["lower_full_name"].isin(dtd["name2"])
]

date_range = pd.date_range("2012-01-01", "2022-05-01")
player_dt_rng = pd.DataFrame(
    list(product(date_range, injured_players.player_id.unique())),
    columns=["game_date", "player_id"],
)


full_date_players = player_dt_rng.merge(
    injured_players, how="left", on=["player_id", "game_date"]
)

full_date_players[
    ["name", "team", "lower_full_name", "full_name"]
] = full_date_players.groupby(["player_id"])[
    ["name", "team", "lower_full_name", "full_name"]
].bfill()

full_date_players[
    ["name", "team", "lower_full_name", "full_name"]
] = full_date_players.groupby(["player_id"])[
    ["name", "team", "lower_full_name", "full_name"]
].ffill()

all_teams = (
    injured_players.groupby(["player_id", "lower_full_name"])["team"]
    .apply(lambda x: list(set(x)))
    .reset_index(name="all_teams")
)

full_date_players = full_date_players.merge(
    all_teams, on=["player_id", "lower_full_name"], how="left"
)

In [23]:
players.game_date.min()

Timestamp('2012-03-28 00:00:00')

In [203]:
dtd_joined = dtd.merge(
    full_date_players[
        ["game_date", "player_id", "lower_full_name", "full_name", "team", "all_teams"]
    ]
    .drop_duplicates(["game_date", "player_id"])
    .rename(columns={"team": "team_x"}),
    left_on=["date", "name"],
    right_on=["game_date", "lower_full_name"],
    how="left",
).merge(
    full_date_players[
        ["game_date", "player_id", "lower_full_name", "full_name", "team", "all_teams"]
    ]
    .drop_duplicates(["game_date", "player_id"])
    .rename(columns={"team": "team_y"}),
    left_on=["date", "name2"],
    right_on=["game_date", "lower_full_name"],
    how="left",
)

In [204]:
dtd_joined["player_id"] = dtd_joined["player_id_x"].fillna(dtd_joined["player_id_y"])
dtd_joined["full_name"] = dtd_joined["full_name_x"].fillna(dtd_joined["full_name_y"])
dtd_joined["team_match"] = (
    (dtd_joined["team"] == dtd_joined["team_x"])
    | (dtd_joined["team"] == dtd_joined["team_y"])
).fillna(False)
dtd_joined["total_same_team"] = dtd_joined.groupby(["id"])["team_match"].transform(sum)
dtd_joined["total_player_id"] = dtd_joined.groupby(["id"])["player_id"].transform(
    "nunique"
)

dtd_joined["all_teams_x"] = dtd_joined["all_teams_x"].apply(
    lambda d: d if isinstance(d, list) else []
)
dtd_joined["all_teams_y"] = dtd_joined["all_teams_y"].apply(
    lambda d: d if isinstance(d, list) else []
)
dtd_joined["career_team_match"] = dtd_joined.apply(
    lambda x: x["team"] in x["all_teams_x"], axis=1
) | dtd_joined.apply(lambda x: x["team"] in x["all_teams_y"], axis=1)

In [205]:
exact_match = dtd_joined.query("total_player_id==1")
no_match = dtd_joined.query("total_player_id==0")
multi_match = dtd_joined.query("total_player_id>1")

In [210]:
# exact_match.shape

In [170]:
# full_date_players.query("lower_full_name=='roberto perez' and game_date=='2020-07-28'")

In [211]:
exact_match_team = (
    multi_match.query("team_match")
    .groupby("id")
    .filter(lambda x: x.player_id.nunique() == 1)
)
dup_team_matches = (
    multi_match.query("team_match")
    .groupby("id")
    .filter(lambda x: x.player_id.nunique() > 1)
)

In [212]:
multi_noteam_match = multi_match[
    ~multi_match.id.isin(exact_match_team.id) & ~multi_match.team_match
]
exact_match_career = (
    multi_noteam_match.query("career_team_match")
    .groupby("id")
    .filter(lambda x: x.player_id.nunique() == 1)
)
dup_career_matches = (
    multi_noteam_match.query("career_team_match")
    .groupby("id")
    .filter(lambda x: x.player_id.nunique() > 1)
)
no_career_matches = multi_noteam_match.groupby("id").filter(
    lambda x: x.career_team_match.sum() == 0
)

In [215]:
final_dtd = (
    pd.concat([exact_match, exact_match_team, exact_match_career])
    .drop(
        columns=[
            "name",
            "name2",
            "game_date_x",
            "player_id_x",
            "lower_full_name_x",
            "full_name_x",
            "team_x",
            "all_teams_x",
            "game_date_y",
            "player_id_y",
            "lower_full_name_y",
            "full_name_y",
            "team_y",
            "all_teams_y",
            "team_match",
            "total_same_team",
            "total_player_id",
            "career_team_match",
        ]
    )
    .rename(columns={"full_name": "name"})
)
final_dtd = final_dtd.sort_values(["player_id", "date"]).reset_index(drop=True)

final_dtd.to_csv("../prosports_data/dtd_2012-2021.csv")

In [43]:
from injury.preprocess.prosports import AlignProsportsMLB

apm = AlignProsportsMLB(dtd, players)
matched_prosports = apm.run()

In [44]:
apm = AlignProsportsMLB(dtd, players)

In [45]:
matched_prosports = apm.run()

### Combine DTD and ILplayers

In [216]:
status_changes = (
    pd.read_csv("statsapi_data/status_changes2012-2021.csv")
    .drop(columns=["resolutionDate", "id"])
    .rename(columns={"description": "notes"})
)

player_names = (
    players[["player_id", "full_name"]]
    .drop_duplicates("player_id")
    .rename(columns={"full_name": "name"})
)


il_df = status_changes[status_changes.notes.str.contains(r"the (\d+)(\s|-)day")]
il_df = il_df.merge(player_names, how="left")
il_df["dtd"] = False

/Users/Sam/opt/miniconda3/envs/statcast/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [217]:
injuries = pd.concat([il_df, final_dtd.drop(columns=["id"])])
injuries["date"] = pd.to_datetime(injuries["date"])
injuries["activated"] = injuries.notes.str.contains("activat")
injuries["transfer"] = injuries.notes.str.contains("transfer")

In [218]:
injuries.reset_index(drop=True).to_pickle("../data/injuries2012-2021.pkl")